## 1. Install and Import Dependencies

### 1.1 Install Dependencies

In [ ]:
!pip install torch torchaudio matplotlib librosa scikit-learn pandas numpy optuna

### 1.2 Load Dependencies

In [ ]:
import os
import csv
import numpy as np
import matplotlib.pyplot as plt
from itertools import groupby
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchaudio
import torchaudio.transforms as T
import optuna
import random

### 1.3 Set random seeds for reproducibility

In [ ]:
torch.manual_seed(42)
np.random.seed(42)
random.seed(42)

### 1.4 Set up GPU/CPU device for PyTorch 

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Data Augmentation function

In [ ]:
class AudioAugmentation:
    """Audio data augmentation techniques for frog call detection"""
    
    def __init__(self, sample_rate=16000):
        self.sample_rate = sample_rate
        
    def add_noise(self, waveform, noise_factor=0.005):
        """Add random Gaussian noise"""
        noise = torch.randn_like(waveform) * noise_factor
        return waveform + noise
    
    def time_shift(self, waveform, shift_limit=0.1):
        """Shift audio in time"""
        shift = int(len(waveform) * shift_limit * (2 * torch.rand(1) - 1))
        if shift > 0:
            return F.pad(waveform[:-shift], (shift, 0))
        elif shift < 0:
            return F.pad(waveform[-shift:], (0, -shift))
        return waveform
    
    def change_speed(self, waveform, speed_factor_range=(0.9, 1.1)):
        """Change playback speed"""
        speed_factor = random.uniform(*speed_factor_range)
        if speed_factor == 1.0:
            return waveform
        
        # Resample to change speed
        original_length = len(waveform)
        new_length = int(original_length / speed_factor)
        
        # Simple linear interpolation for speed change
        indices = torch.linspace(0, original_length - 1, new_length)
        indices_floor = torch.floor(indices).long()
        indices_ceil = torch.ceil(indices).long()
        
        # Clamp indices
        indices_floor = torch.clamp(indices_floor, 0, original_length - 1)
        indices_ceil = torch.clamp(indices_ceil, 0, original_length - 1)
        
        # Linear interpolation
        alpha = indices - indices_floor.float()
        interpolated = (1 - alpha) * waveform[indices_floor] + alpha * waveform[indices_ceil]
        
        # Pad or truncate to original length
        if len(interpolated) > original_length:
            return interpolated[:original_length]
        else:
            return F.pad(interpolated, (0, original_length - len(interpolated)))
    
    def change_volume(self, waveform, volume_range=(0.5, 1.5)):
        """Change volume"""
        volume_factor = random.uniform(*volume_range)
        return waveform * volume_factor
    
    def apply_augmentation(self, waveform, augment_prob=0.5):
        """Apply random augmentations"""
        if random.random() < augment_prob:
            # Apply random combination of augmentations
            augmentations = [
                lambda x: self.add_noise(x),
                lambda x: self.time_shift(x),
                lambda x: self.change_speed(x),
                lambda x: self.change_volume(x)
            ]
            
            # Randomly select 1-2 augmentations
            selected = random.sample(augmentations, random.randint(1, 2))
            for aug in selected:
                waveform = aug(waveform)
        
        return waveform

## 3. Audio file loading and pre-processing before training

In [ ]:
class FrogCallDataset(Dataset):
    def __init__(self, pos_dir, neg_dir, max_length=8000, augment=False, augment_prob=0.5):
        self.max_length = max_length
        self.augment = augment
        self.augmenter = AudioAugmentation() if augment else None
        self.augment_prob = augment_prob
        self.files = []
        self.labels = []
        
        # Load positive samples (frog calls)
        if os.path.exists(pos_dir):
            for file in os.listdir(pos_dir):
                if file.endswith(('.wav', '.WAV')):
                    self.files.append(os.path.join(pos_dir, file))
                    self.labels.append(1)
        
        # Load negative samples (no frog calls)
        if os.path.exists(neg_dir):
            for file in os.listdir(neg_dir):
                if file.endswith(('.wav', '.WAV')):
                    self.files.append(os.path.join(neg_dir, file))
                    self.labels.append(0)
    
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        # Load audio
        wav = self.load_wav_16k_mono(self.files[idx])
        label = self.labels[idx]
        
        # Apply augmentation if enabled and this is training
        if self.augment and self.augmenter:
            wav = self.augmenter.apply_augmentation(wav, self.augment_prob)
        
        # Preprocess
        spectrogram, label = self.preprocess(wav, label)
        return spectrogram, torch.tensor(label, dtype=torch.float32)
    
    def load_wav_16k_mono(self, filename):
        """Load a WAV file, convert it to mono and resample to 16kHz"""
        waveform, sample_rate = torchaudio.load(filename)
        
        # Convert to mono if stereo
        if waveform.shape[0] > 1:
            waveform = torch.mean(waveform, dim=0, keepdim=True)
        
        # Resample to 16kHz if needed
        if sample_rate != 16000:
            resampler = T.Resample(sample_rate, 16000)
            waveform = resampler(waveform)
        
        return waveform.squeeze(0)
    
    def preprocess(self, wav, label):
        # Truncate or pad to max_length (0.5s = 8000 samples at 16kHz)
        if len(wav) > self.max_length:
            wav = wav[:self.max_length]
        else:
            padding = self.max_length - len(wav)
            wav = F.pad(wav, (padding, 0), value=0.0)
        
        # Create spectrogram using STFT
        stft = torch.stft(
            wav, 
            n_fft=256,
            hop_length=64,
            return_complex=True
        )
        spectrogram = torch.abs(stft)
        
        # Add channel dimension
        spectrogram = spectrogram.unsqueeze(0)
        
        return spectrogram, label

## 4. Deffines structure of residual blocks with skip connections

In [ ]:
class ResidualBlock(nn.Module):
    """Residual block for audio CNN"""
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        
        # Shortcut connection
        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )
    
    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


## 5. Deffines ARCHITECTURE FOR BINARY DETECTION

In [ ]:
class FrogCallCNN(nn.Module):
    def __init__(self, input_shape, num_blocks=[2, 2, 2], dropout_rate=0.5):
        super(FrogCallCNN, self).__init__()
        
        # Initial convolution
        self.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        # Residual layers
        self.layer1 = self._make_layer(64, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(64, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(128, 256, num_blocks[2], stride=2)
        
        # Global average pooling
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        
        # Classifier
        self.dropout = nn.Dropout(dropout_rate)
        self.fc1 = nn.Linear(256, 128)
        self.fc2 = nn.Linear(128, 1)
        
    def _make_layer(self, in_channels, out_channels, num_blocks, stride):
        layers = []
        layers.append(ResidualBlock(in_channels, out_channels, stride))
        for _ in range(1, num_blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        
        x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        
        return x

## 6. Deffines Early stopping parameters

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=7, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_loss = None
        self.counter = 0
        self.best_weights = None
        
    def __call__(self, val_loss, model):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.save_checkpoint(model)
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
            self.save_checkpoint(model)
        else:
            self.counter += 1
            
        if self.counter >= self.patience:
            if self.restore_best_weights:
                model.load_state_dict(self.best_weights)
            return True
        return False
    
    def save_checkpoint(self, model):
        """Saves model when validation loss decrease."""
        self.best_weights = model.state_dict().copy()


## 7. Deffines the training function for the model

In [ ]:
# 7. Training Function

def train_model(model, train_loader, val_loader, epochs=50, lr=0.001, model_type='CNN', 
                patience=10, min_delta=0.001, confidence_threshold=0.9):
    
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=5, factor=0.5)
    early_stopping = EarlyStopping(patience=patience, min_delta=min_delta)
    
    history = {
        'train_loss': [], 'val_loss': [],
        'train_accuracy': [], 'val_accuracy': [],
        'train_precision': [], 'val_precision': [],
        'train_recall': [], 'val_recall': [],
        'train_f1': [], 'val_f1': []
    }
    
    print(f"Training {model_type} model with early stopping...")
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_correct = 0
        train_total = 0
        train_tp, train_fp, train_fn = 0, 0, 0
        
        for batch_idx, (data, targets) in enumerate(train_loader):
            data, targets = data.to(device), targets.to(device)
            
            optimizer.zero_grad()
            outputs = model(data).squeeze()
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            
            # Calculate metrics
            predicted = (outputs > confidence_threshold).float()
            train_total += targets.size(0)
            train_correct += (predicted == targets).sum().item()
            
            train_tp += ((predicted == 1) & (targets == 1)).sum().item()
            train_fp += ((predicted == 1) & (targets == 0)).sum().item()
            train_fn += ((predicted == 0) & (targets == 1)).sum().item()
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_correct = 0
        val_total = 0
        val_tp, val_fp, val_fn = 0, 0, 0
        
        with torch.no_grad():
            for data, targets in val_loader:
                data, targets = data.to(device), targets.to(device)
                outputs = model(data).squeeze()
                loss = criterion(outputs, targets)
                val_loss += loss.item()
                
                predicted = (outputs > confidence_threshold).float()
                val_total += targets.size(0)
                val_correct += (predicted == targets).sum().item()
                
                val_tp += ((predicted == 1) & (targets == 1)).sum().item()
                val_fp += ((predicted == 1) & (targets == 0)).sum().item()
                val_fn += ((predicted == 0) & (targets == 1)).sum().item()
        
        # Calculate epoch metrics
        avg_train_loss = train_loss / len(train_loader)
        avg_val_loss = val_loss / len(val_loader)
        
        train_accuracy = 100 * train_correct / train_total
        val_accuracy = 100 * val_correct / val_total
        
        train_precision = train_tp / (train_tp + train_fp) if (train_tp + train_fp) > 0 else 0
        train_recall = train_tp / (train_tp + train_fn) if (train_tp + train_fn) > 0 else 0
        train_f1 = 2 * (train_precision * train_recall) / (train_precision + train_recall) if (train_precision + train_recall) > 0 else 0
        
        val_precision = val_tp / (val_tp + val_fp) if (val_tp + val_fp) > 0 else 0
        val_recall = val_tp / (val_tp + val_fn) if (val_tp + val_fn) > 0 else 0
        val_f1 = 2 * (val_precision * val_recall) / (val_precision + val_recall) if (val_precision + val_recall) > 0 else 0
        
        # Store history
        history['train_loss'].append(avg_train_loss)
        history['val_loss'].append(avg_val_loss)
        history['train_accuracy'].append(train_accuracy)
        history['val_accuracy'].append(val_accuracy)
        history['train_precision'].append(train_precision)
        history['val_precision'].append(val_precision)
        history['train_recall'].append(train_recall)
        history['val_recall'].append(val_recall)
        history['train_f1'].append(train_f1)
        history['val_f1'].append(val_f1)
        
        # Learning rate scheduling
        scheduler.step(avg_val_loss)
        
        print(f'Epoch {epoch+1}/{epochs}:')
        print(f'  Train Loss: {avg_train_loss:.4f}, Val Loss: {avg_val_loss:.4f}')
        print(f'  Train Acc: {train_accuracy:.2f}%, Val Acc: {val_accuracy:.2f}%')
        print(f'  Train F1: {train_f1:.4f}, Val F1: {val_f1:.4f}')
        print(f'  Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
        
        # Early stopping
        if early_stopping(avg_val_loss, model):
            print(f'Early stopping triggered after epoch {epoch+1}')
            break
    
    return history

## 8. Deffines function for hyperparameter tuning for optimization

In [ ]:
def hyperparameter_tuning(input_shape, train_loader, val_loader, n_trials=20):
    
    study = optuna.create_study(direction='maximize')
    study.optimize(lambda trial: objective(trial, input_shape, train_loader, val_loader), 
                  n_trials=n_trials)
    
    print("Best trial:")
    print(f"  Value: {study.best_trial.value}")
    print(f"  Params: {study.best_trial.params}")
    
    return study.best_trial.params

## 9. Deffines objective function for hyperparameter tuning

In [ ]:
def objective(trial, input_shape, train_loader, val_loader):
    """Objective function for hyperparameter optimization"""
    
    # Suggest hyperparameters
    lr = trial.suggest_float('lr', 1e-5, 1e-2, log=True)
    dropout_rate = trial.suggest_float('dropout_rate', 0.2, 0.7)
    
    model = FrogCallCNN(input_shape, dropout_rate=dropout_rate).to(device)
    
    # Train with suggested parameters
    history = train_model(model, train_loader, val_loader, epochs=20, lr=lr, patience=5)
    
    # Return the best validation F1 score
    return max(history['val_f1'])

## 10. Deffines function for graphical representations for tracking model training performance

In [ ]:
def plot_training_history(history, title="Training History"):
    fig, ((ax1, ax2), (ax3, ax4), (ax5, ax6)) = plt.subplots(3, 2, figsize=(15, 12))
    
    # Loss
    ax1.plot(history['train_loss'], 'r', label='Train Loss')
    ax1.plot(history['val_loss'], 'b', label='Val Loss')
    ax1.set_title('Loss')
    ax1.legend()
    ax1.grid(True)
    
    # Accuracy
    ax2.plot(history['train_accuracy'], 'r', label='Train Accuracy')
    ax2.plot(history['val_accuracy'], 'b', label='Val Accuracy')
    ax2.set_title('Accuracy (%)')
    ax2.legend()
    ax2.grid(True)
    
    # Precision
    ax3.plot(history['train_precision'], 'r', label='Train Precision')
    ax3.plot(history['val_precision'], 'b', label='Val Precision')
    ax3.set_title('Precision')
    ax3.legend()
    ax3.grid(True)
    
    # Recall
    ax4.plot(history['train_recall'], 'r', label='Train Recall')
    ax4.plot(history['val_recall'], 'b', label='Val Recall')
    ax4.set_title('Recall')
    ax4.legend()
    ax4.grid(True)
    
    # F1 Score
    ax5.plot(history['train_f1'], 'r', label='Train F1')
    ax5.plot(history['val_f1'], 'b', label='Val F1')
    ax5.set_title('F1 Score')
    ax5.legend()
    ax5.grid(True)
    
    # Final metrics summary
    ax6.text(0.5, 0.5, f'Final Metrics:\nTrain Acc: {history["train_accuracy"][-1]:.2f}%\nVal Acc: {history["val_accuracy"][-1]:.2f}%\nVal F1: {history["val_f1"][-1]:.4f}', 
             transform=ax6.transAxes, ha='center', va='center', fontsize=12,
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    ax6.set_title('Final Performance')
    ax6.axis('off')
    
    plt.suptitle(title)
    plt.tight_layout()
    plt.show()

## 11. Deffines function for loading recordings

In [ ]:
def predict_on_long_audio(model, wav, window_size=8000, hop_size=4000, confidence_threshold=0.9):
    """Make predictions on a long audio file using sliding window with specified confidence threshold"""
    model.eval()
    predictions = []
    confident_predictions = []
    
    for start in range(0, len(wav) - window_size + 1, hop_size):
        segment = wav[start:start + window_size]
        spectrogram = preprocess_audio_segment(segment).to(device)
        
        with torch.no_grad():
            pred = model(spectrogram).item()
            predictions.append(pred)
            # Only count as positive if above confidence threshold
            confident_predictions.append(pred > confidence_threshold)
    
    return predictions, confident_predictions

## 12. Deffines model training function

In [ ]:
# 11. Main Execution Script

def train_model(confidence_threshold=0.9):
    # Define paths
    POS_DIR = '/Users/jasminenelson/Library/CloudStorage/OneDrive-Personal/masters_thesis/Library/frog'
    NEG_DIR = '/Users/jasminenelson/Library/CloudStorage/OneDrive-Personal/masters_thesis/Library/nofrog'
   
 # Create base dataset without augmentation to get the split indices
    print("Loading frog call dataset...")
    base_dataset = FrogCallDataset(POS_DIR, NEG_DIR, max_length=8000, augment=False)
    
    if len(base_dataset) == 0:
        print("Warning: No audio files found. Please check your data paths.")
        return
    
    # Get input shape
    sample_spec, _ = base_dataset[0]
    input_shape = sample_spec.shape
    print(f"Spectrogram input shape: {input_shape}")
    
    # Split dataset (80% train, 20% val)
    dataset_size = len(base_dataset)
    train_size = int(0.8 * dataset_size)
    val_size = dataset_size - train_size
    
    # Get the split indices
    train_indices, val_indices = torch.utils.data.random_split(range(dataset_size), [train_size, val_size])
    
    # Create separate datasets for training (with augmentation) and validation (without)
    train_dataset = FrogCallDataset(POS_DIR, NEG_DIR, max_length=8000, augment=True, augment_prob=0.6)
    val_dataset = FrogCallDataset(POS_DIR, NEG_DIR, max_length=8000, augment=False)
    
    # Create subset datasets using the indices
    train_subset = torch.utils.data.Subset(train_dataset, train_indices.indices)
    val_subset = torch.utils.data.Subset(val_dataset, val_indices.indices)
    
    # Create data loaders
    train_loader = DataLoader(train_subset, batch_size=32, shuffle=True)
    val_loader = DataLoader(val_subset, batch_size=32, shuffle=False)
    
    print(f"Training samples: {len(train_subset)}")
    print(f"Validation samples: {len(val_subset)}")
    
    # Hyperparameter tuning
    print("\nHyperparameter tuning")
    best_params = hyperparameter_tuning(input_shape, train_loader, val_loader, n_trials=10)
    
    # Create and train model
    if best_params:
        print(f"Using optimized parameters: {best_params}")
        model = FrogCallCNN(
            input_shape, 
            dropout_rate=best_params.get('dropout_rate', 0.5)
        ).to(device)
        lr = best_params.get('lr', 0.001)
    else:
        print("Using default parameters...")
        model = FrogCallCNN(input_shape, dropout_rate=0.5).to(device)
        lr = 0.001
    
    print(f"\nModel architecture:")
    print(model)
    print(f"Total parameters: {sum(p.numel() for p in model.parameters()):,}")

# Train model with 90% confidence threshold
    print(f"\nTraining ResNet model with {confidence_threshold*100}% confidence threshold...")
    history = train_model(model, train_loader, val_loader, epochs=100, lr=lr, 
                         patience=15, confidence_threshold=confidence_threshold)
    
    # Plot results
    plot_training_history(history)
    
    # Save model
    torch.save(model.state_dict(), 'frog_call_cnn_final.pth')
    print("ANALYSIS COMPLETE")

## 13. Defines function to apply model to real data

In [ ]:
def test_site(confidence_threshold=0.9):

    excel_file_path = '/Users/jasminenelson/Library/CloudStorage/OneDrive-Personal/masters_thesis/site_files/Excel/Turmalina_24_F/Turmalina_2024_021.xlsx'
   
    input_shape = (1, 129, 125)
    
    # Load model
    model = FrogCallCNN(input_shape=input_shape, dropout_rate=0.5).to(device)
    model.load_state_dict(torch.load('frog_call_cnn_final.pth', map_location=device))
    model.eval()
    print(f"\nTesting model with {confidence_threshold*100}% confidence threshold...")
    
    # Read the Excel file
    try:
        df = pd.read_excel(excel_file_path)
        print(f"Loaded Excel file: {excel_file_path}")
        print(f"Found {len(df)} files to process")
    except Exception as e:
        print(f"Error reading Excel file: {e}")
        return
    
    # Initialize result columns
    df['frog_calls'] = None
    df['avg_confidence'] = None
    df['max_confidence'] = None
    df['total_segments'] = None
    df['high_confidence_segments'] = None
    df['percentage_with_frogs'] = None
    df['confidence_threshold'] = confidence_threshold
    
    processed_count = 0
    successful_count = 0
    
    # Process each file in the Excel sheet
    for index, row in df.iterrows():
        filename = row['filename']
        file_path = row['file_path']
        
        print(f"Processing {filename}...")
        processed_count += 1
        
        # Construct full file path
        full_path = file_path
        
        # Check if file exists
        if not os.path.exists(full_path):
            print(f"  File not found: {full_path}")
            df.at[index, 'frog_calls'] = 'File Not Found'
            continue
        
        try:
            # Load audio file
            wav = load_mp3_16k_mono(full_path)
            
            if wav is not None:
                predictions, confident_predictions = predict_on_long_audio(
                    model, wav, confidence_threshold=confidence_threshold
                )
                
                if predictions:
                    # Calculate statistics
                    avg_confidence = np.mean(predictions)
                    max_confidence = np.max(predictions)
                    total_segments = len(predictions)
                    
                    # Determine if frog calls are present
                    frog_present = any(confident_predictions)
                    frog_calls = "Present" if frog_present else "Absent"
                    
                    # Count segments with high confidence frog calls
                    high_confidence_segments = sum(confident_predictions)
                    percentage_with_frogs = (high_confidence_segments / total_segments) * 100
                    
                    # Add results to dataframe
                    df.at[index, 'frog_calls'] = frog_calls
                    df.at[index, 'avg_confidence'] = round(avg_confidence, 4)
                    df.at[index, 'max_confidence'] = round(max_confidence, 4)
                    df.at[index, 'total_segments'] = total_segments
                    df.at[index, 'high_confidence_segments'] = high_confidence_segments
                    df.at[index, 'percentage_with_frogs'] = round(percentage_with_frogs, 2)
                    
                    successful_count += 1
                    print(f"  {filename}: {frog_calls} (avg={avg_confidence:.3f}, max={max_confidence:.3f}, {high_confidence_segments}/{total_segments} segments)")
                else:
                    print(f"  Could not process {filename} - no predictions generated")
                    df.at[index, 'frog_calls'] = 'Processing Error'
            else:
                print(f"  Could not load {filename}")
                df.at[index, 'frog_calls'] = 'Load Error'
                
        except Exception as e:
            print(f"  Error processing {filename}: {e}")
            df.at[index, 'frog_calls'] = f'Error: {str(e)}'
    
    # Save the updated Excel file
    try:
        # Create output filename with threshold info
        base_name = os.path.splitext(excel_file_path)[0]
        output_file = f"{base_name}_results_aug_{int(confidence_threshold*100)}pct2.xlsx"
        
        df.to_excel(output_file, index=False)
        print(f"\nResults saved to: {output_file}")
        
        # Also save a backup of the original with results
        backup_file = f"{base_name}_results2_aug.xlsx"
        df.to_excel(backup_file, index=False)
        print(f"Backup saved to: {backup_file}")
        
    except Exception as e:
        print(f"Error saving Excel file: {e}")
        # Fallback to CSV if Excel save fails
        csv_file = f"{base_name}_results_{int(confidence_threshold*100)}pct.csv"
        df.to_csv(csv_file, index=False)
        print(f"Results saved as CSV: {csv_file}")
    
    # Print summary
    print(f"\nSummary with {confidence_threshold*100}% confidence threshold:")
    print(f"Files processed: {processed_count}")
    print(f"Files successfully analyzed: {successful_count}")
    
    if successful_count > 0:
        files_with_frogs = len(df[df['frog_calls'] == 'Present'])
        detection_rate = (files_with_frogs / successful_count) * 100
        print(f"Files with frog calls detected: {files_with_frogs}")
        print(f"Detection rate: {detection_rate:.1f}%")
        
        # Show some statistics
        valid_results = df[df['frog_calls'].isin(['Present', 'Absent'])]
        if len(valid_results) > 0:
            print(f"\nConfidence Statistics:")
            print(f"Average confidence across all files: {valid_results['avg_confidence'].mean():.3f}")
            print(f"Maximum confidence found: {valid_results['max_confidence'].max():.3f}")
            print(f"Average percentage with frogs: {valid_results['percentage_with_frogs'].mean():.1f}%")
    
    print("\n" + "="*50)
    print("ANALYSIS COMPLETE")
    print("="*50)
    
    return df

## 14. Run model training function

In [ ]:
if __name__ == "__main__":
  train_model()

## 15. Run trained model on real data

In [ ]:
if __name__ == "__main__":
    test_site()